In [ ]:
# Script to create date for Time Dimesion

In [52]:
# Import required libraries
import sys
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, expr, current_date, date_format

In [30]:
# Date Source data generating function for next 1 year
def _date_dim(start_run_dt: str = '20230101') -> list:
    _data = []
    _start_date = datetime.strptime(start_run_dt, '%Y%m%d')
    _data.append([
        datetime.strftime(_start_date, '%Y-%m-%d'), 
        datetime.strftime(_start_date, '%d'),
        datetime.strftime(_start_date, '%m'),
        datetime.strftime(_start_date, '%Y'),
        datetime.strftime(_start_date, '%A')])
    _next_date = _start_date
    for i in range(0, 364):
        _next_date = _next_date + timedelta(days = 1)
        _data.append([
        datetime.strftime(_next_date, '%Y-%m-%d'), 
        datetime.strftime(_next_date, '%d'),
        datetime.strftime(_next_date, '%m'),
        datetime.strftime(_next_date, '%Y'),
        datetime.strftime(_next_date, '%A')])
        
    return _data

# Column for Date Source
_cols = ['date', 'day', 'month', 'year', 'day_of_week']

In [32]:
# Generate the Spark Session
spark = SparkSession \
    .builder \
    .appName("Date DIM Load") \
    .enableHiveSupport() \
    .getOrCreate()

print("Spark Session Created, webui: " + spark.sparkContext.uiWebUrl)

:: loading settings :: url = jar:file:/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d61a3196-3945-4e66-b445-6d7e0d4e9b55;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.1 in central
	found io.delta#delta-storage;2.1.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 237ms :: artifacts dl 13ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	io.delta#delta-core_2.12;2.1.1 from central in [default]
	io.delta#delta-storage;2.1.1 from central in [default]
	org.antlr#antlr4-runtime;4.

Spark Session Created, webui: http://46346aae9d54:4040


In [63]:
# Create the source dataframe
df_landing = spark.createDataFrame(data=_date_dim(), schema=_cols)

# Transform columns
df_stage = df_landing \
    .withColumn("date", to_date("date", "yyyy-MM-dd")) \
    .withColumn("day", expr("cast(day as int)")) \
    .withColumn("month", expr("cast(month as int)")) \
    .withColumn("year", expr("cast(year as int)")) \
 
# Create final dim df
df_dim = df_stage \
    .withColumn("row_wid", date_format("date", "yyyyMMdd")) \
    .withColumn("insert_dt", current_date()) \
    .withColumn("update_dt", current_date()) \
    


In [64]:
# Write data in DIM DATE
print("SPARK_APP: Writing data to DIM_DATE in APPEND MODE")

df_dim.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("dw.dim_date")

SPARK_APP: Writing data to DIM_DATE in APPEND MODE


In [69]:
spark.sql("select count(1) as dim_date_count from dw.dim_date").show()

+--------------+
|dim_date_count|
+--------------+
|           365|
+--------------+



In [70]:
print("SPARK_APP: Dimension Date Loaded Successfully")

SPARK_APP: Dimension Date Loaded Successfully


In [71]:
spark.stop()